# flox: Faster GroupBy aggregations with Xarray

**Authors** Deepak Cherian, Kevin Sampson, Matthew Rocklin

Significantly faster groupby calculations are now possible through a new-ish package in the Xarray/Dask/Pangeo ecosystem called [flox](https://flox.readthedocs.io/en/latest/). Practically, this means faster climatologies, faster resampling, faster histogramming, and faster compositing of array datasets.


## The National Water Model Dataset


To show off, we demonstrate county-wise aggregation of output from the National Water Model (NWM) available on the [AWS Public Data Registry](https://registry.opendata.aws/nwm-archive/).


Quoting [the NOAA page](https://water.noaa.gov/about/nwm) for more.

> The National Water Model (NWM) is a hydrologic modelling framework that simulates observed and forecast streamflow over the entire continental United States (CONUS). The NWM simulates the water cycle with mathematical representations of the different processes and how they fit together. This complex representation of physical processes such as snowmelt and infiltration and movement of water through the soil layers varies significantly with changing elevations, soils, vegetation types and a host of other variables. Additionally, extreme variability in precipitation over short distances and times can cause the response on rivers and streams to change very quickly. Overall, the process is so complex that to simulate it with a mathematical model means that it needs a very high powered computer or supercomputer in order to run in the time frame needed to support decision makers when flooding is threatened.  

> All CONUS model configurations provide streamflow for 2.7 million river reaches and other hydrologic information on 1km and 250m grids.



## Problem description

We want to calculate county-level means for 3 hourly time series data on the 250m grid. This is a *Groupby* problem. 

GroupBy is a term used for a very common analysis pattern commonly called "split-apply-combine" ([Wickham, 2011](https://www.jstatsoft.org/article/view/v040i01)) wherein an analyst 
- *Splits* a dataset into groups (e.g. counties),
- *Applies* a transformation to each group of data (here a reduction like `.mean`)
- *Combines* the results of `apply` to form a new dataset


For this problem we will split the dataset into counties, apply the `mean`, and then combine the results back.

With [Xarray](https://docs.xarray.dev/en/stable/user-guide/groupby.html), this would look like
```python
dataset.groupby(counties).mean()
```

However Xarray's default algorithm is a simple for-loop over groups and doesn't work very well for large distributed problems.

## Enter `flox`.

`flox` solves a long-standing problem in the Pangeo array computing ecosytem of computing GroupBy reductions. It implements a parallel groupby algorithm (using a tree reduction) to substantially improve performance of groupby reductions with dask. 
  - Specifically, `flox` speeds up [reduction methods](https://flox.readthedocs.io/en/latest/aggregations.html) like `groupby(...).mean()`, `groupby(...).max()`, etc, but not `groupby.map`.
  - `flox` also significantly speeds up groupby reductions with pure numpy arrays using optimized implementations in the [`numpy-groupies` package](https://github.com/ml31415/numpy-groupies).
  - `flox` allows more complicated groupby operations such as lazy grouping by a dask array, and grouping by multiple variables. Use `flox.xarray.xarray_reduce` for [these operations](https://flox.readthedocs.io/en/latest/xarray.html). Xarray currently only supports grouping by a single numpy variable.

See [here](https://flox.readthedocs.io/en/latest/intro.html) for short examples.

### How do I use it?

Run `mamba install flox` and `xarray>=2022.06.0` will use it by default for `.groupby`, `.groupby_bins`, and `.resample`!

A lot of effort was spent in ensuring backwards compatibility, so your workloads should only work better. Let us know if it [does not](https://github.com/pydata/xarray/issues)


## Setup cluster with Coiled

To demonstrate calculation on a cloud-available dataset, we will use [Coiled](https://coiled.io) to set up a dask cluster in AWS `us-east-1`.

In [1]:
import coiled

cluster = coiled.Cluster(n_workers=2, backend_options={"region_name": "us-east-1"})
cluster.adapt(minimum=2, maximum=40)
client = cluster.get_client()

Output()

Output()

2023-07-03 16:42:09,520 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=2 maximum=40


In [2]:
client

Connection method: Cluster object,Cluster type: coiled.Cluster
Dashboard: https://cluster-rrxia.dask.host/XfcS_VqtMnjhf1mW/status,
Dashboard: https://cluster-rrxia.dask.host/XfcS_VqtMnjhf1mW/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tls://10.0.83.16:8786,Workers: 0
Dashboard: http://10.0.83.16:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Setup

In [3]:
%load_ext watermark

import flox  # make sure its available
import fsspec
import numpy as np
import rioxarray
import xarray as xr

xr.set_options(
    display_expand_attrs=False,
    display_expand_coords=False,
    display_expand_data=True,
)

%watermark -iv

xarray   : 2023.6.0
fsspec   : 2023.6.0
numpy    : 1.25.0
flox     : 0.7.2
rioxarray: 0.14.1
coiled   : 0.7.9



## Load NWM data

In [4]:
ds = xr.open_zarr(
    fsspec.get_mapper("s3://noaa-nwm-retrospective-2-1-zarr-pds/rtout.zarr", anon=True),
    consolidated=True,
)
ds

<xarray.Dataset>
Dimensions:       (time: 122479, y: 15360, x: 18432)
Coordinates: (3)
Data variables:
    crs           |S1 ...
    sfcheadsubrt  (time, y, x) float64 dask.array<chunksize=(224, 350, 350), meta=np.ndarray>
    zwattablrt    (time, y, x) float64 dask.array<chunksize=(224, 350, 350), meta=np.ndarray>
Attributes: (7)

In [5]:
with open('/Users/dcherian/work/xarray.dev/src/posts/flox/ds-repr.html', mode="w") as file:
   file.write(ds._repr_html_())

Each field in this dataset is big!

In [6]:
with open('/Users/dcherian/work/xarray.dev/src/posts/flox/zwattablrt-repr.html', mode="w") as file:
   file.write(ds.zwattablrt._repr_html_())

ds.zwattablrt

<xarray.DataArray 'zwattablrt' (time: 122479, y: 15360, x: 18432)>
dask.array<open_dataset-zwattablrt, shape=(122479, 15360, 18432), dtype=float64, chunksize=(224, 350, 350), chunktype=numpy.ndarray>
Coordinates: (3)
Attributes: (4)

We'll subset to a single year subset for demo purposes

In [7]:
subset = ds.zwattablrt.sel(time=slice("2001-01-01", "2001-12-31"))
with open('/Users/dcherian/work/xarray.dev/src/posts/flox/subset-repr.html', mode="w") as file:
   file.write(subset._repr_html_())
subset

<xarray.DataArray 'zwattablrt' (time: 2920, y: 15360, x: 18432)>
dask.array<getitem, shape=(2920, 15360, 18432), dtype=float64, chunksize=(224, 350, 350), chunktype=numpy.ndarray>
Coordinates: (3)
Attributes: (4)

## Load county raster for grouping

A raster TIFF file identifying counties by a unique integer was created separately and saved. 

We load that using [rioxarray](https://corteva.github.io/rioxarray/html/rioxarray.html)

In [8]:
counties = rioxarray.open_rasterio(
    "s3://nwm-250m-us-counties/Counties_on_250m_grid.tif", chunks="auto"
).squeeze()

# remove any small floating point error in coordinate locations
_, counties_aligned = xr.align(ds, counties, join="override")

with open('/Users/dcherian/work/xarray.dev/src/posts/flox/counties-repr.html', mode="w") as file:
   file.write(counties_aligned._repr_html_())
counties_aligned

<xarray.DataArray (y: 15360, x: 18432)>
dask.array<getitem, shape=(15360, 18432), dtype=int32, chunksize=(1820, 18432), chunktype=numpy.ndarray>
Coordinates: (4)
Attributes: (9)

We'll need the unique county IDs later, calculate that now.

In [9]:
county_id = np.unique(counties_aligned.data).compute()
county_id = county_id[county_id != 0]
print(f"There are {len(county_id)} counties!")

There are 3108 counties!


## GroupBy with flox

We could run the computation as

```python
subset.groupby(counties_aligned).mean()
```

This would use flox in the background.

However it would also load `counties_aligned` in to memory (an unfortunate Xarray implementation detail) which is not so bad (only a gig). To avoid egress charges, we'll instead go through `flox.xarray` which allows you to lazily groupby a dask array (here `counties_aligned`) as long as you pass in the expected group labels in `expected_groups`.

See [here](https://flox.readthedocs.io/en/latest/intro.html#with-dask) for more.

In [10]:
import flox.xarray

county_mean = flox.xarray.xarray_reduce(
    subset,
    counties_aligned.rename("county"),
    func="mean",
    expected_groups=(county_id,),
)

with open('/Users/dcherian/work/xarray.dev/src/posts/flox/county-mean-repr.html', mode="w") as file:
   file.write(county_mean._repr_html_())
county_mean

<xarray.DataArray 'zwattablrt' (time: 2920, county: 3108)>
dask.array<groupby_nanmean, shape=(2920, 3108), dtype=float64, chunksize=(224, 3108), chunktype=numpy.ndarray>
Coordinates: (4)
Attributes: (4)

In [ ]:
county_mean.load()

/Users/dcherian/mambaforge/envs/coiled/lib/python3.11/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 9.57 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.DataArray 'zwattablrt' (time: 2920, county: 3108)>
array([[1.77092178, 1.79430538, 1.71125074, ..., 1.97485099, 1.90160141,
        1.86238405],
       [1.77233924, 1.79459704, 1.71276979, ..., 1.97483864, 1.90030852,
        1.86265516],
       [1.77146351, 1.7946418 , 1.71316782, ..., 1.97485716, 1.89992493,
        1.8630656 ],
       ...,
       [1.62511916, 1.4306925 , 1.41918174, ..., 1.98066828, 1.96049339,
        1.94494132],
       [1.62372207, 1.43174076, 1.42150228, ..., 1.98081146, 1.96038362,
        1.94497373],
       [1.62567311, 1.43302003, 1.42396092, ..., 1.98076333, 1.96061008,
        1.94486356]])
Coordinates: (4)
Attributes: (4)

The computation proceeds very nicely. 

We don't anticipate trouble scaling this computation up to the full dataset.

## Cleanup

In [10]:
cluster.shutdown()

2023-07-03 15:54:43,862 - distributed.deploy.adaptive_core - INFO - Adaptive stop
2023-07-03 15:54:44,272 - distributed.deploy.adaptive_core - INFO - Adaptive stop


## Summary

[flox](https://flox.readthedocs.io) makes many large Groupby problems tractable! Use it.


[flox](https://flox.readthedocs.io) also makes many small but more complicated (e.g. multiple variables) Groupby problems tractable! Use it.

We [anticipate](https://github.com/pydata/xarray/issues/6610) upgrading Xarray's interface to enable more complicated GroupBy computations. In the mean time, use flox!